# Feature Slection by Random Search with XGBoost with Solar Power Ramp Events

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

## Importing the dataset

In [2]:
os.getcwd()
os.chdir('../datasets')
# dataset = pd.read_csv('Social_Network_Ads.csv')
dataset = pd.read_csv('rampdata66.csv')
# X = dataset.iloc[:, [2, 3]].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 66].values
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V58,V59,V60,V61,V62,V63,V64,V65,V66,Class
0,0.001610,0.001852,0.445164,0.710982,0.482693,0.346400,0.786719,0.296912,0.212884,0.414958,...,4,2,2,2,2,2,1,2,2,4
1,0.127630,0.085961,0.548310,0.907729,0.999993,0.607910,0.184483,0.370349,0.051022,0.696017,...,2,2,2,2,2,4,4,2,2,2
2,0.034725,0.109288,0.510308,0.922363,0.999993,0.598586,0.159467,0.365388,0.054600,0.720091,...,2,2,2,2,2,2,2,2,2,2
3,0.033372,0.276992,0.473963,0.927030,0.999993,0.604930,0.140143,0.369037,0.092530,0.730662,...,4,4,4,2,2,2,2,2,4,4
4,0.171606,0.153857,0.456074,0.928770,0.999993,0.595534,0.121081,0.371288,0.086637,0.736614,...,2,4,4,2,2,2,2,4,4,4


## Encoding categorical data

In [3]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# labelencoder_X = LabelEncoder()
# X[ : , 1] = labelencoder_X.fit_transform(X[ : , 1])
# X

## Splitting the dataset into the Training set and Test set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Cross-validation for XGBoosting: 
2folds for each of 100 candidates, totalling 200 fits, (it takes time about 5mins)

https://scikit-learn.org/stable/modules/model_evaluation.html


{'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 500}

In [6]:
# from sklearn import metrics   ##Additional scklearn functions
# from sklearn.model_selection import cross_validate
# from sklearn.model_selection import GridSearchCV   #Perforing grid search

# from xgboost.sklearn import XGBClassifier
# XGB_model = XGBClassifier()

# test_params = {'n_estimators':[100, 200, 300, 400, 500], 'learning_rate':[0.05, 0.1, 0.2, 0.3, 0.5], 
#                'max_depth':[2,4,8,12]}

# grid = GridSearchCV(estimator = XGB_model, param_grid = test_params, n_jobs=4, 
#                     scoring='f1_weighted',cv=2, verbose=3)

# grid.fit(X_train,y_train)
# print(grid.best_params_)

### Fitting XGB  
Evaluate metric(s) by cross-validation and also record fit/score times.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

https://scikit-learn.org/stable/modules/cross_validation.html

In [7]:
from sklearn import metrics   ##Additional scklearn functions
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from xgboost.sklearn import XGBClassifier
params = {'n_estimators': 500, 'max_depth': 12, 'learning_rate': 0.3}
classifier = XGBClassifier(**params, criterion = 'entropy', random_state=0)
cv_results = cross_validate(classifier, X, y, cv=2)
sorted(cv_results.keys())                          

['fit_time', 'score_time', 'test_score', 'train_score']

## Random Search for Feature Selection

In [25]:
from sklearn.model_selection import cross_val_score

result = []# Number of iterations
# N_search = 300# Random seed initialization
N_search = 300# Random seed initialization

np.random.seed(1)
for i in range(N_search):
    # Generate a random number of featurest
    N_columns =  list(np.random.choice(range(X.shape[1]),1)+1)
    
    # Given the number of features, generate features without replacement
    columns = list(np.random.choice(range(X.shape[1]), N_columns, replace=False))
    
    # Perform k-fold cross validation
#     scores = cross_val_score(classifier,X[:,columns], y, cv=2, scoring="accuracy")
#     scores = cross_val_score(classifier,X[:,columns], y, cv=2, scoring="f1_micro")
    scores = cross_val_score(classifier,X[:,columns], y, cv=2, scoring="f1_weighted")


    # Store the result
    result.append({'columns':columns,'performance':np.mean(scores)})
    # Sort the result array in descending order for performance measure
result.sort(key=lambda x : -x['performance'])

C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn

In [26]:
result[0]
# X[:,result[0]['columns']]
print(result[0]['columns'])

[35, 20, 7, 29, 51, 50, 41, 43, 26, 40, 13, 55, 16, 60, 64, 42, 30, 61, 0, 34, 8, 23, 27, 14, 44, 38, 39, 59, 5, 45, 11, 2, 1, 47, 46, 36, 54, 52, 24, 21, 22, 32, 10, 58, 33, 4]


In [27]:
cv_results

C:\Users\Mhdella\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([5.21102405, 5.55015445]),
 'score_time': array([0.07878923, 0.08377576]),
 'test_score': array([0.85639687, 0.87558808]),
 'train_score': array([1., 1.])}

In [28]:
sum(y_train.shape+y_test.shape)

3828

## Single metric evaluation using cross_validate

In [29]:
sorted(cv_results.keys())                         
['fit_time', 'score_time', 'test_score']
cv_results['test_score']    

array([0.85639687, 0.87558808])

## Making the Confusion Matrix
#### We can use this value to calculate the new performance measure with this subset of the features.

In [30]:
from sklearn.model_selection import cross_val_predict
# y_pred = cross_val_predict(classifier, X, y, cv=2)
y_pred = cross_val_predict(classifier, X[:,result[0]['columns']], y, cv=2)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y)
cm

array([[  63,   29,    0,    2],
       [  65, 1064,    1,  177],
       [   0,    0,    9,    3],
       [   3,  197,   21, 2194]], dtype=int64)

In [31]:
print('True Hi_Ramps=',cm[0,0]+cm[2,2])
print('False Hi_Ramps=',cm[1,0]+cm[2,0]+cm[3,0]+cm[0,2]+cm[1,2]+cm[3,2])

True Hi_Ramps= 72
False Hi_Ramps= 90


In [32]:
from sklearn.metrics import f1_score
f1_score(y, y_pred, average=None)

array([0.56      , 0.81940701, 0.41860465, 0.91588395])

####  Multiple metric evaluation using cross_validate 

In [33]:
# scores = cross_validate(classifier, X, y, cv=2, scoring=('f1_weighted'), return_train_score=True)
scores = cross_validate(classifier, X[:,result[0]['columns']], y, cv=2, scoring=('f1_weighted'), 
                           return_train_score=True)

## Analysis on the outcomes y, (classes of solar ramp events)

In [34]:
y.shape
len(y)
print('C1=', len(y[y==1]),'C2=', len(y[y==2]),
      'C3=', len(y[y==3]),'C4=', len(y[y==4]))

C1= 131 C2= 1290 C3= 31 C4= 2376
